In [2]:
#简单理解LSTM神经网络https://blog.csdn.net/shijing_0214/article/details/52081301/
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

C:\Programmingtools\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
#载入数据集
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

#输入图片是28*28
n_inputs = 28 #输入层有28个神经元，对应着图片的一行有28个像素，每次输入图片的一行到神经网络中
max_time = 28 #图片像素一共28行，总共输入28次
lstm_size = 100 #memory block(类似隐藏单元)
n_classes = 10 #0-9共10个分类
batch_size = 50 #每批次50个样本
n_batch = mnist.train.num_examples // batch_size #计算一共多少个批次

#这里的none表示第一个维度可以是任意的长度
x = tf.placeholder(tf.float32,[None,784])
#正确的标签
y = tf.placeholder(tf.float32,[None,10])

#只需定义一个权值和偏度值，其余的tensorflow会自动帮我们初始化
#初始化权值
weights = tf.Variable(tf.truncated_normal([lstm_size, n_classes], stddev=0.1))
#tf.truncated_normal与tf.random_normal的区别https://blog.csdn.net/u014687582/article/details/78027061
#初始化偏度值
biases = tf.Variable(tf.constant(0.1, shape=[n_classes]))
#tensorflow之常用函数（tf.Constant）http://blog.sina.com.cn/s/blog_e504f4340102yd4k.html

#定义LSTM网络
def LSTM(X,weights,biases):
    #inputs = [batch_size, max_time, n_inputs]
    inputs = tf.reshape(X, [-1,max_time,n_inputs]) #-1代表50，因为None
    #定义LSTM基本CELL
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size) #定义100个BasicLSTMCell
#   final_state[state, batch_size, cell.state.size]
#   state有两个值，分别是0和1，0代表cell_state,1代表hidden_state
#   batch_size是50，代表每批次50个样本
#   cell.state.size是100，代表有100个隐藏单元
#   final_state[0]是cell_state (block中间的信号)
#   final_state[1]是hidden_state (最后的输出信号)
#   final_state[1]只能记录最后一次输出的结果，总共有28个序列，outputs记录着序列每一次输出的结果，max_time从0到27,outputs分别对应不同序列的输出

#   outputs: The RNN output `Tensor`.
#       If time_major == False (default), this will be a `Tensor` shaped:
#           `[batch_size, max_time, cell.output_size]`.
#       If time_major == True, this will be a `Tensor` shaped:
#           `[max_time, batch_size, cell.output_size]`.
#   有三个维度，与time_major有关，batch_size = 50，max_time = 28，cell.output_size = 100
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1],weights) + biases) #final_state[1]代表整个神经网络的输出
    return results
#计算LSTM的返回结果
prediction = LSTM(x, weights, biases)
#损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))#把correct_prediction变为float32类型
#初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
            
        acc = sess.run(accury,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter" + str(epoch) + ",Testing Accuracy+" + str(acc))
